# Fake news detection

**Authors:** Peter Mačinec, Simona Miková

## Model construction

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%load_ext tensorboard
%tensorboard --logdir logs --bind_all

import sys
sys.path.append('..')

from src.models.fasttext import read_fasttext_model

In [2]:
df = pd.read_csv('../data/preprocessed/dataset.csv', index_col=0)

In [2]:
%%time
fasttext = read_fasttext_model('../models/fasttext/wiki-news-300d-1M.vec')

CPU times: user 1min 41s, sys: 1.88 s, total: 1min 43s
Wall time: 1min 43s


In [13]:
len(fasttext), len(fasttext['work'])

(999995, 300)

In [11]:
fasttext['work']

array([-0.072 ,  0.013 , -0.0777, -0.0491, -0.085 , -0.0337, -0.0243,
        0.0472, -0.0575, -0.0356,  0.0931, -0.0579, -0.0667, -0.0612,
       -0.0634, -0.052 ,  0.1209, -0.0841,  0.0455,  0.0819, -0.0535,
        0.075 ,  0.0889,  0.1769,  0.0675, -0.0347, -0.0773, -0.0488,
       -0.0569,  0.0227,  0.0035,  0.0395,  0.0183,  0.0542, -0.0188,
        0.0203, -0.0252, -0.0498,  0.0278,  0.0521,  0.0046, -0.0992,
        0.0415, -0.034 ,  0.0507, -0.0419, -0.0607,  0.0656,  0.0076,
        0.0696, -0.0558,  0.0456, -0.6356, -0.0703, -0.1587, -0.1104,
        0.055 ,  0.016 ,  0.0074,  0.0279, -0.0508,  0.0394, -0.0134,
       -0.0315, -0.0289, -0.0604,  0.0277,  0.017 , -0.0156,  0.0355,
        0.0201, -0.0229,  0.1151, -0.0961,  0.0129,  0.0488,  0.0038,
       -0.0607,  0.0469,  0.1148,  0.0382,  0.0008, -0.0133, -0.1429,
        0.0423, -0.0841, -0.0146, -0.0523,  0.0353, -0.0013,  0.1078,
        0.0076,  0.0498, -0.0432, -0.0121, -0.054 ,  0.0992,  0.0559,
       -0.0699,  0.0

In [23]:
model = tf.keras.Sequential()
# Add an Embedding layer expecting input vocab of size 1000, and
# output embedding dimension of size 64.
# self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim, 
# embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
# trainable=False)


model.add(layers.Embedding(
    input_dim=len(lines-1),
    output_dim=300,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False
))

# Add a LSTM layer with 128 internal units.
model.add(layers.LSTM(128))

# Add a Dense layer with 10 units and softmax activation.
model.add(layers.Dense(10, activation='softmax'))

model.summary()


TypeError: unsupported operand type(s) for -: 'list' and 'int'

In [ ]:
import tensorflow.keras as keras
from src.models.model import FakeNewsDetectionNet

model = FakeNewsDetectionNet(
    dim_input=len(fasttext),
    dim_embeddings=len(fasttext['work']),
    dim_output=2
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

callbacks = [
    keras.callbacks.TensorBoard(
        log_dir=os.path.join("logs", timestamp()),
        histogram_freq=1,
        profile_batch=0
    )
]

model.fit(
    x=train_images,
    y=train_labels,
    batch_size=16,
    validation_data=(test_images, test_labels),
    callbacks=callbacks,
    epochs=30
)

model.summary()